In [ ]:
import os
import pickle
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

# --- Configuration ---
CLIENT_SECRETS_FILE = os.getenv("client_secret.json")  # Download this from 
TOKEN_PICKLE_FILE = os.getenv("TOKEN_PICKLE_FILE", "token.pickle")

SCOPES = ["https://www.googleapis.com/auth/youtube"] # Full access, needed for deletion
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"
TOKEN_PICKLE_FILE = "token.pickle"

# List of video IDs you want to remove from your Watch Later playlist
VIDEO_IDS_TO_REMOVE = [
    "HOLyfxEHQic",
    "Qahx1dPlH28",
    # Add more video IDs here
]

def get_authenticated_service():
    """Authenticates and returns a YouTube API service object."""
    credentials = None
    # Token pickle file stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists(TOKEN_PICKLE_FILE):
        with open(TOKEN_PICKLE_FILE, "rb") as token:
            credentials = pickle.load(token)

    # If there are no (valid) credentials available, let the user log in.
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(google.auth.transport.requests.Request())
        else:
            flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES
            )
            # Open a browser for authentication
            credentials = flow.run_local_server(port=0) # port=0 will find an available port
        # Save the credentials for the next run
        with open(TOKEN_PICKLE_FILE, "wb") as token:
            pickle.dump(credentials, token)

    return googleapiclient.discovery.build(
        API_SERVICE_NAME, API_VERSION, credentials=credentials
    )

def get_watch_later_playlist_id(youtube):
    """
    Retrieves the Watch Later playlist ID for the authenticated user.
    Tries to get it directly from relatedPlaylists, otherwise constructs it from the channel ID.
    """
    try:
        print("Fetching channel details to find/construct Watch Later playlist ID...")
        response = youtube.channels().list(
            part="contentDetails,id",  # Request 'id' for channel ID, 'contentDetails' for relatedPlaylists
            mine=True
        ).execute()

        if not response.get("items"):
            print("Error: Could not find channel items for the authenticated user in API response.")
            return None

        channel_data = response["items"][0]
        api_channel_id_raw = channel_data.get("id") # Get the channel ID

        if not api_channel_id_raw:
            print("Error: Channel ID not found in API response.")
            return None
        
        api_channel_id = api_channel_id_raw.strip() # Clean up potential whitespace
        print(f"Raw Channel ID from API: '{api_channel_id_raw}', Stripped Channel ID: '{api_channel_id}'")

        # Try to get watchLater directly from relatedPlaylists
        # .get() is used for safe dictionary access
        related_playlists = channel_data.get("contentDetails", {}).get("relatedPlaylists", {})
        watch_later_playlist_id_direct = related_playlists.get("watchLater")

        if watch_later_playlist_id_direct:
            # If found directly, use it (and strip any potential whitespace)
            final_watch_later_id = watch_later_playlist_id_direct.strip()
            print(f"Found Watch Later playlist ID directly from relatedPlaylists: {final_watch_later_id}")
            return final_watch_later_id
        else:
            # If 'watchLater' key is not found or is empty, construct it
            print(f"'watchLater' key was missing or empty in relatedPlaylists. Constructing ID from Channel ID: {api_channel_id}")
            
            base_channel_id_for_wl = api_channel_id 
            if api_channel_id.startswith("UC"):
                print(f"Channel ID '{api_channel_id}' starts with 'UC'. Slicing off 'UC' prefix.")
                base_channel_id_for_wl = api_channel_id[2:] # Remove "UC" prefix
            else:
                # This case is less common for user channels but handles other ID formats
                print(f"Channel ID '{api_channel_id}' does not start with 'UC'. Using it as is for base.")
            
            constructed_watch_later_id = "WL" + base_channel_id_for_wl
            print(f"Base ID for WL: '{base_channel_id_for_wl}', Constructed Watch Later playlist ID: {constructed_watch_later_id}")
            return constructed_watch_later_id

    except googleapiclient.errors.HttpError as e:
        print(f"An API error occurred while fetching channel details: {e}")
        return None
    except Exception as e: # Catch any other unexpected error during parsing
        # It's good to log the type of exception and traceback in a real app
        print(f"An unexpected error occurred while processing channel details: {type(e).__name__} - {e}")
        return None

def get_playlist_items_to_remove(youtube, playlist_id, video_ids_to_remove_set):
    """
    Fetches all items from a playlist and returns a list of playlistItemIds
    for videos that are in the video_ids_to_remove_set.
    """
    playlist_item_ids_for_deletion = []
    video_ids_found_in_playlist = set()
    next_page_token = None

    print(f"\nFetching items from playlist ID: {playlist_id}...")

    while True:
        try:
            request = youtube.playlistItems().list(
                part="snippet,id",  # 'id' gives playlistItemId, 'snippet' gives videoId
                playlistId=playlist_id,
                maxResults=50, # Max allowed per page
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                video_id = item["snippet"]["resourceId"]["videoId"]
                playlist_item_id = item["id"] # This is the ID needed for deletion

                if video_id in video_ids_to_remove_set:
                    print(f"  Found video to remove: '{item['snippet']['title']}' (Video ID: {video_id}, PlaylistItemID: {playlist_item_id})")
                    playlist_item_ids_for_deletion.append(playlist_item_id)
                    video_ids_found_in_playlist.add(video_id)

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break # No more pages

        except googleapiclient.errors.HttpError as e:
            print(f"An API error occurred while fetching playlist items: {e}")
            # Depending on the error, you might want to retry or break
            if e.resp.status == 404: # Playlist not found
                print(f"Error: Playlist with ID {playlist_id} not found.")
            break # Stop processing this playlist on error

    # Report videos that were in VIDEO_IDS_TO_REMOVE but not found in the playlist
    not_found_in_playlist = video_ids_to_remove_set - video_ids_found_in_playlist
    if not_found_in_playlist:
        print("\nVideos specified for removal but NOT found in the Watch Later playlist:")
        for vid in not_found_in_playlist:
            print(f"  - {vid}")

    return playlist_item_ids_for_deletion

def remove_items_from_playlist(youtube, playlist_item_ids):
    """Removes a list of playlistItemIds from their playlist."""
    if not playlist_item_ids:
        print("\nNo items to remove.")
        return

    print("\nStarting removal process...")
    for item_id in playlist_item_ids:
        try:
            youtube.playlistItems().delete(id=item_id).execute()
            print(f"  Successfully removed playlist item: {item_id}")
        except googleapiclient.errors.HttpError as e:
            print(f"  Failed to remove playlist item {item_id}: {e}")
            # You might want to add more sophisticated error handling here,
            # e.g., if the item was already removed or if there's a quota issue.

def main():
    if not os.path.exists(CLIENT_SECRETS_FILE):
        print(f"Error: {CLIENT_SECRETS_FILE} not found. Please download it from Google Cloud Console.")
        return

    if not VIDEO_IDS_TO_REMOVE:
        print("No video IDs specified in VIDEO_IDS_TO_REMOVE. Exiting.")
        return

    print("Starting YouTube Watch Later cleanup script...")
    youtube = get_authenticated_service()
    if not youtube:
        print("Failed to authenticate. Exiting.")
        return

    watch_later_id = get_watch_later_playlist_id(youtube)
    if not watch_later_id:
        print("Could not determine Watch Later playlist ID. Exiting.")
        return

    # Use a set for efficient lookup of video IDs to remove
    video_ids_to_remove_set = set(VIDEO_IDS_TO_REMOVE)

    playlist_item_ids_to_delete = get_playlist_items_to_remove(
        youtube,
        watch_later_id,
        video_ids_to_remove_set
    )

    if playlist_item_ids_to_delete:
        remove_items_from_playlist(youtube, playlist_item_ids_to_delete)
        print("\nRemoval process completed.")
    else:
        print("\nNo matching videos found in Watch Later to remove.")

if __name__ == "__main__":
    main()

Starting YouTube Watch Later cleanup script...
Fetching channel details to find/construct Watch Later playlist ID...
Raw Channel ID from API: 'UCwHoqnNsoYQbI13oy6s8HHQ', Stripped Channel ID: 'UCwHoqnNsoYQbI13oy6s8HHQ'
'watchLater' key was missing or empty in relatedPlaylists. Constructing ID from Channel ID: UCwHoqnNsoYQbI13oy6s8HHQ
Channel ID 'UCwHoqnNsoYQbI13oy6s8HHQ' starts with 'UC'. Slicing off 'UC' prefix.
Base ID for WL: 'wHoqnNsoYQbI13oy6s8HHQ', Constructed Watch Later playlist ID: WLwHoqnNsoYQbI13oy6s8HHQ

Fetching items from playlist ID: WLwHoqnNsoYQbI13oy6s8HHQ...
An API error occurred while fetching playlist items: <HttpError 404 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=snippet%2Cid&playlistId=WLwHoqnNsoYQbI13oy6s8HHQ&maxResults=50&alt=json returned "The playlist identified with the request's <code>playlistId</code> parameter cannot be found.". Details: "[{'message': "The playlist identified with the request's <code>playlistId</code> para